# LLM Locales, HuggingFace y BERT

# BERT

Los modelos de lenguaje fueron creados para todo tipo de tareas NLP, de hecho desde su concepción en la tesis de [Attention is all you need](https://arxiv.org/abs/1706.03762) los modelos de lenguaje fueron creados como un modelos no generativos.

Hay modelos de lenguaje para todo tipo de tareas como:

1. Reconocimiento de Entidades
2. Análisis de Sentimiento
3. Preguntas y Respuestas
4. Traducción
5. Resumir
6. Generar Embeddings

[BERT](https://huggingface.co/blog/bert-101) a pesar de ser de los primeros modelos de lenguaje (transformer), sus variantes siguen siendo las opciones más utilizadas para cualquier tarea que no sea generación de texto, donde principalmente usan arquitecturas basadas en GPT.

Muchos de los experimentos en los que hemos trabajado usan algún modelo de lenguaje no generativo para complementar al modelo generativo. Son modelos especializados, más pequeños, eficientes y que fácilmente los podemos ejecutar on-prem.

In [ ]:
from huggingface_hub import notebook_login
from transformers import pipeline
import pandas as pd
import torch


# Hugging Face

Hugging Face es otra herramienta indispensable para trabajar con modelos de lenguaje locales ya sea generativos o no generativos.

En estos ejercicios voy a explicarles cómo me apoyo de la plataforma para llevar a cabo nuestros experimentos.

Lo primero que tenemos que hacer es [crear una cuenta](https://huggingface.co/join) y [generar un token](https://huggingface.co/settings/tokens) para poder descargar los modelos desde nuestro notebook.

Palabras Clave:

* Transformers
* Inference
* Pipeline
* Task


In [ ]:
notebook_login()

### Generación de Texto

Es con lo que hemos trabajado hasta ahorita, lo que hizo popular a los modelos de lenguaje. Son los modelos más pesados y desarrollar con ellos de manera local es casi imposible para el hardware de consumo general.

## Tasks

In [ ]:
gpt_generator = pipeline("text-generation", model = "meta-llama/Llama-3.2-1B")
print(gpt_generator("Mi nombre es Juan y tengo ", max_length=20))

### Reconocimiento de Entidades (NER)

Identificación de personas, organizaciones, lugares, etc.

In [ ]:
classifier = pipeline("ner", model = "iiiorg/piiranha-v1-detect-personal-information")
classifier("Mi nombre es Juan y vivo en Sinaloa.")

### Análisis de Sentimiento

Modelos especializados en detectar emociones y opiniones.

In [ ]:
sentiment_task = pipeline("sentiment-analysis", model="clapAI/modernBERT-base-multilingual-sentiment", tokenizer="clapAI/modernBERT-base-multilingual-sentiment")
sentiment_task("no me gusta")

### Mask

El modelado de lenguaje enmascarado es la tarea de ocultar algunas de las palabras en una oración y predecir qué palabras deberían reemplazar esas máscaras.

Este tipo de modelos nunca los he usado, creo que su tarea es muy específica y su uso más práctico es muy parecido a un modelo generativo, sin ser autorregresivo.

Para este ejemplo voy a aprovechar y voy a usar una API un poquito de más bajo nivel para explicar otros conceptos.

In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM

model_id = "answerdotai/ModernBERT-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForMaskedLM.from_pretrained(model_id)

text = "The capital of France is [MASK]."
inputs = tokenizer(text, return_tensors="pt")
outputs = model(**inputs)

# To get predictions for the mask:
masked_index = inputs["input_ids"][0].tolist().index(tokenizer.mask_token_id)
predicted_token_id = outputs.logits[0, masked_index].argmax(axis=-1)
predicted_token = tokenizer.decode(predicted_token_id)
print("Token ID Predecido:", predicted_token_id)
print("Token Predecido:", predicted_token)
print("Tokens Totales:", len(outputs.logits[0, masked_index]))

### Salida y Top_K

En este caso el modelo de lenguaje nos regresa las probabilidades del token que se encuentra en masked_index, que es el índice de nuestra máscara. **[MASK]** es un token especial para este task de BERT, así como los tokens de tools, pensamiento o de formato de chat.

Con esta salida podemos hacer una selección usando algún algoritmo de selección, como los que ya conocemos como temperatura, top_k o top_p.



In [ ]:
# Obtener las probabilidades de los tokens en la posición de la máscara
mask_logits = outputs.logits[0, masked_index]


# Convierte logits a probabilidades usando softmax
probabilities = torch.softmax(mask_logits, dim=-1)
top_k = 5

print(f"\nTop {top_k} predicciones con probabilidad [MASK]:")
top_k_prob, top_k_indices = probabilities.topk(top_k)

for i in range(5):
    token = tokenizer.decode(top_k_indices[i])
    percentage = top_k_prob[i].item() * 100
    print(f"Token: {token}, Percentage: {percentage:.2f}%")


Este modelo nos da como salida la probabilidad de todos los tokens de su vocabulario, es responsabilidad nuestra elegir cuál token queremos.

Los modelos generativos como Gemini son muy parecidos a esto, pero solo pueden predecir basándose en el texto que se encuentra antes del último token.

In [ ]:
# Saca una lista de todos los tokens y todas sus probabilidades
all_token_ids = torch.arange(probabilities.size(0))
all_tokens = [tokenizer.decode(token_id) for token_id in all_token_ids]
all_probabilities = probabilities.tolist()

data = []
for token, prob in zip(all_tokens, all_probabilities):
    data.append({'Token': token, 'probabilidad': prob, '%': f'{(prob * 100):.3f}%'})

df_all_tokens = pd.DataFrame(data)

df_all_tokens_sorted = df_all_tokens.sort_values(by='%', ascending=False).reset_index(drop=True)

print("\nDataframe con todos los tokens, probabilidades, y porcentajes:")
df_all_tokens_sorted


### Ejercicio

¿Qué hacen los modelos que responden a preguntas, si no son generativos?

Modifica el siguiente código con un modelo y el texto necesario.

In [ ]:
qa_model = pipeline("question-answering", model=)
question = ""
context = "Culiacán is the largest city in Sinaloa with a population of approximately 1 million people in its metropolitan area. Culiacán is the capital city of Sinaloa state in northwestern Mexico. "
qa_model(question = question, context = context)